#### JDBC connection string

Integrated Security (Windows authentication i.e. without username and password) and download JDBC driver for reference [click here](https://learn.microsoft.com/en-us/sql/connect/jdbc/connecting-with-ssl-encryption?view=sql-server-ver16)

using java version - `JDK 20`\
using os version - `windows11`\
Put `mssql-jdbc_auth-12.4.1.x64.dll` in `jdk/bin/`\
If needs put downloaded dll, jar and jre in `spark/jars`\
If needs enable TCP/IP for SQL server -> search on windows for `mmc` i.e. SQL server configuration manager -> add snap in for SQL server Network Configuration -> Protocols for MSSQL-> enable TCP/IP


In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\spark\\spark-3.4.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession.builder \
    .appName('Spark - SQL Server Integrated Authentication Example') \
    .master('local') \
    .getOrCreate()

In [3]:
DATABASE_NAME='LiveClassAssignment'
PORT='63739'

In [4]:
connectionUrl = f'''
jdbc:sqlserver://localhost:{PORT};
databaseName={DATABASE_NAME};
integratedSecurity=true;
encrypt=true;
trustServerCertificate=true;
'''

connectionUrl = "".join(line.strip() for line in connectionUrl.splitlines())

```sql
SELECT Department,AVG(Salary) as gross_average_salary FROM EmployeeDetails e
LEFT JOIN EmployeeBonus eb on e.Employee_id=eb.Employee_ref_id_FK
GROUP BY Department
```


In [5]:
def get_db_table_data(tableName:str):
    return spark.read.format("jdbc") \
    .option("url", connectionUrl) \
    .option("dbtable", tableName) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

In [6]:
emp_details_df =get_db_table_data('EmployeeDetails')
emp_bonus_df=get_db_table_data('EmployeeBonus')

In [15]:
from pyspark.sql.functions import avg,count,max,min,monotonically_increasing_id

# mssql_df.groupBy('Department')\
#     .agg(avg('salary').alias('average_salary'))\
#     .show(truncate=False)

salary_by_department = emp_details_df.join(emp_bonus_df,emp_details_df['Employee_id']==emp_bonus_df['Employee_ref_id_FK'],'left')\
    .groupBy('Department')\
    .agg(avg('salary').alias('average_salary'),
         count("*").alias("no_of_employees"),
         max('salary').alias('maximum_salary'),
         min('salary').alias('minimum_salary'),
         )\
             .withColumn("id", monotonically_increasing_id())
    # .show(truncate=False)
salary_by_department.show()

+----------+------------------+---------------+--------------+--------------+---+
|Department|    average_salary|no_of_employees|maximum_salary|minimum_salary| id|
+----------+------------------+---------------+--------------+--------------+---+
|        HR|166666.66666666666|              3|        300000|        100000|  0|
|     Admin|          250000.0|              5|        500000|         80000|  1|
|   Account|          137500.0|              2|        200000|         75000|  2|
+----------+------------------+---------------+--------------+--------------+---+



In [11]:
#TODO: need to fix this to write data to table
def load_df_to_db(df):
    mode='overwrite'
    # properties={"driver":"com.microsoft.sqlserver.jdbc.SQLServerDriver"}
    tableName='departmental_salary'
    df.write \
    .format("jdbc") \
    .mode(mode)\
    .option("url", connectionUrl) \
    .option("dbtable", tableName) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [12]:
load_df_to_db(salary_by_department)

AttributeError: 'DataFrameWriter' object has no attribute 'withColumn'

In [ ]:
spark.stop()